In [1]:
import pymongo
import math 
# import re
import pandas as pd
# import numpy as np
import pylab 
from matplotlib import pyplot as plt
# from scipy import stats 
from  qgrid import show_grid as show 
from datetime import datetime
# import seaborn as sns

def gettimetamps(containerId): 
    containerdata=list(sensors.find({'target':containerId},projection=['timestamp']))
    begintime= containerdata[0]['timestamp']
    endtime=containerdata[-1]['timestamp']
    return begintime , endtime 


def getpowers(begin, end): 
    #get the power consumption of the system  between begin and end 
    x=list(sensors.find({'target':'system','timestamp' :{'$gte':begin,'$lte':end}},projection=['rapl','timestamp']))
    conso= pd.DataFrame(x)
    sonde=next(iter(x[0]['rapl']['0']))
    conso['power']=conso['rapl'].apply(lambda row :math.ldexp( row['0'][sonde]['RAPL_ENERGY_PKG'],-32))
    return conso.loc[:,['timestamp','power']]

def getenergy(powers):
    return powers['power'].sum()

def getmeasures(testname,databasename='rapls2'): 
    db=client[databasename]
    meausres = db['recap'+testname].find(projection={'_id': False,'id':False})
    return pd.DataFrame(list(meausres))

client = pymongo.MongoClient('172.16.45.8', 27017)

# simple example

In [2]:
getmeasures("test2160119")

,execution energy,execution time,name,warmup energy,warmup time
0,181.449402,7,t2,0.0,0
1,506.409241,21,t2,0.0,0
2,892.012817,27,t3,0.0,0


# more details 

## getting the sensors collection 

In [67]:
testname="test2160119" # the mahcine used for the test 
client = pymongo.MongoClient('172.16.45.8', 27017) # the Ip addresse of the database
db = client.rapls2 # name of the database [dbname in startmachine.sh ]
sensors= db["sensortest2160119"]

sensors.create_index([('timestamp',pymongo.ASCENDING)]) # just to accelerate the search requests 

'timestamp_1'

## getting the timestamps of the container 

In [77]:
mycontainerId='8e085cf136075aee9b0a2b722fac86e77b84f4fcc2079b9c004fae023b3ee9dd' # the id of the container 
# you can get this id using the following command in the test machine 
# docker inspect --format "{{.Id}}" containername
beginTime , endTime =gettimetamps(mycontainerId)

print("begin time {} \nend time  {}".format(beginTime,endTime))

begin time 2019-01-16 19:15:22.291000 
end time  2019-01-16 19:15:27.293000


## getting the power a container 

In [79]:
power = getpowers(beginTime,endTime) # get an array of the power consumption during the time stamp 
print('my container powers')
power


my container powers


,timestamp,power
0,2019-01-16 19:15:22.291,26.662781
1,2019-01-16 19:15:23.292,30.913391
2,2019-01-16 19:15:24.292,23.523682
3,2019-01-16 19:15:25.292,23.526733
4,2019-01-16 19:15:26.292,23.543640
5,2019-01-16 19:15:27.293,23.524963


## getting the total energy 

In [20]:
totalenergy= getenergy(power) # to get the energy consumption of the container 

print(str(totalenergy) + ' joules')

885.20361328125 joules
